## A catalog of [data algebra expression](https://github.com/WinVector/data_algebra) methods.


In [13]:
import datetime
import os
import warnings
import numpy as np
import pandas as pd
import pickle
import gzip

from data_algebra.data_ops import *
import data_algebra.SQLite
import data_algebra.BigQuery
import data_algebra.PostgreSQL
import data_algebra.SparkSQL
import data_algebra.MySQL
from data_algebra.parse_by_lark import parse_by_lark
import data_algebra.test_util
import data_algebra.util

In [14]:
def mk_example():
    datetime_format = "%Y-%m-%d %H:%M:%S"
    date_format = "%Y-%m-%d"
    d = pd.DataFrame({
        'row_id': [0, 1, 2, 3],
        'a': [False, False, True, True],
        'b': [False, True, False, True],
        'q': [1, 1, 2, 2],
        'x': [.1, .2, .3, .4],
        'y': [2.4, 1.33, 1.2, 1.1],
        'z': [1.6, None, -2.1, numpy.nan],
        'g': ['a', 'a', 'b', 'ccc'],
        's2': ['z', 'q', '11', 'b'],
        "str_datetime_col": ["2000-01-01 12:13:21", "2020-04-05 14:03:00", "2000-01-01 12:13:21", "2020-04-05 14:03:00"],
        "str_date_col": ["2000-03-01", "2020-04-05", "2000-03-01", "2020-04-05"],
        "datetime_col_0": pd.to_datetime(
            pd.Series(["2010-01-01 12:13:21", "2030-04-05 14:03:00", "2010-01-01 12:13:21", "2030-04-05 14:03:00"]),
            format=datetime_format,
        ),
        "datetime_col_1": pd.to_datetime(
            pd.Series(["2010-01-01 12:11:21", "2030-04-06 14:03:00", "2010-01-01 12:11:21", "2030-04-06 14:03:00"]),
            format=date_format,
        ),
        "date_col_0": pd.to_datetime(
            pd.Series(["2000-01-02", "2035-04-05", "2000-01-02", "2035-04-05"]),
            format=date_format
        ).dt.date,
        "date_col_1": pd.to_datetime(
            pd.Series(["2000-01-02", "2035-05-05", "2000-01-02", "2035-05-05"]),
            format=date_format
        ).dt.date,
    })
    return d


def f(expression):
    return (
        descr(d=d)
            .extend({'new_column': expression})
            .select_columns(['row_id', 'new_column'])
            .order_rows(['row_id'])
    )


def fg(expression):
    return (
        descr(d=d)
            .extend(
                {'new_column': expression},
                partition_by=['g'])
            .select_columns(['g', 'row_id', 'new_column'])
            .order_rows(['g', 'row_id'])
    )


def fp(expression):
    return (
        descr(d=d)
            .project(
                {'new_column': expression},
                group_by=['g'])
            .order_rows(['g'])
    )


def fw(expression):
    return (
        descr(d=d)
            .extend(
                {'new_column': expression},
                partition_by=['g'],
                order_by=['row_id'])
            .select_columns(['g', 'row_id', 'new_column'])
            .order_rows(['g', 'row_id'])
    )


In [15]:
d = mk_example()

d


,row_id,a,b,q,x,y,z,g,s2,str_datetime_col,str_date_col,datetime_col_0,datetime_col_1,date_col_0,date_col_1
0,0,False,False,1,0.1,2.40,1.6,a,z,2000-01-01 12:13:21,2000-03-01,2010-01-01 12:13:21,2010-01-01 12:11:21,2000-01-02,2000-01-02
1,1,False,True,1,0.2,1.33,NaN,a,q,2020-04-05 14:03:00,2020-04-05,2030-04-05 14:03:00,2030-04-06 14:03:00,2035-04-05,2035-05-05
2,2,True,False,2,0.3,1.20,-2.1,b,11,2000-01-01 12:13:21,2000-03-01,2010-01-01 12:13:21,2010-01-01 12:11:21,2000-01-02,2000-01-02
3,3,True,True,2,0.4,1.10,NaN,ccc,b,2020-04-05 14:03:00,2020-04-05,2030-04-05 14:03:00,2030-04-06 14:03:00,2035-04-05,2035-05-05


,row_id,a,b,q,x,y,z,g,s2,str_datetime_col,str_date_col,datetime_col_0,datetime_col_1,date_col_0,date_col_1
0,0,False,False,1,0.1,2.40,1.6,a,z,2000-01-01 12:13:21,2000-03-01,2010-01-01 12:13:21,2010-01-01 12:11:21,2000-01-02,2000-01-02
1,1,False,True,1,0.2,1.33,NaN,a,q,2020-04-05 14:03:00,2020-04-05,2030-04-05 14:03:00,2030-04-06 14:03:00,2035-04-05,2035-05-05
2,2,True,False,2,0.3,1.20,-2.1,b,11,2000-01-01 12:13:21,2000-03-01,2010-01-01 12:13:21,2010-01-01 12:11:21,2000-01-02,2000-01-02
3,3,True,True,2,0.4,1.10,NaN,ccc,b,2020-04-05 14:03:00,2020-04-05,2030-04-05 14:03:00,2030-04-06 14:03:00,2035-04-05,2035-05-05


In [16]:
data_def = {k: v for (k, v) in descr(d=d).column_map().items()}

def parse(exp):
    return parse_by_lark(exp, data_def=data_def)


In [17]:
expressions = [
    'x + y',
    'x - y',
    'row_id // q',
    'row_id % q',
    'x / y',
    'x * y',
    'x ** y',
    'x == y',
    'x > y',
    'x >= y',
    'x < y',
    'x <= y',
    'x != y',
    '-x',
    'not a',
    'a & b',
    'a and b',
    'a | b',
    'a or b',
    'z.sign()',
    'x.sin()',
    'x.cos()',
    'x.arcsin()',
    'x.arccos()',
    'x.arctan()',
    'x.arctan2(y)',
    'x.sinh()',
    'x.cosh()',
    'x.tanh()',
    'x.arcsinh()',
    'x.arccosh()',
    'x.arctanh()',
    'y.floor()',
    'z.floor()',
    'y.ceil()',
    'z.ceil()',
    'x.sum()',
    'x.exp()',
    'y.expm1()',
    'x.log()',
    'x.log10()',
    'x.log1p()',
    'row_id.mod(2)',
    'row_id.remainder(2)',
    'x.sqrt()',
    'z.abs()',
    'row_id.maximum(x)',
    'row_id.minimum(x)',
    'row_id.fmax(x)',
    'row_id.fmin(x)',
    'y.round()',
    'y.around(2)',
    'z.is_null()',
    'z.is_bad()',
    'a.if_else(x, y)',
    'row_id.is_in({1, 3})',
    'g.concat(s2)',
    'g %+% "_" %+% s2',
    'z.coalesce(2)',
    'z %?% 2',
    'z.coalesce_0()',
    'g.mapv({"a": 1, "b": 2, "z": 26}, 0)',
    'y.as_int64()',
    'y.as_str()',
    'g.trimstr(0, 2)',
    'datetime_col_0.datetime_to_date()',
    'str_date_col.parse_date()',
    'str_datetime_col.parse_datetime()',
    'datetime_col_0.format_datetime()',
    'date_col_0.format_date()',
    'date_col_0.dayofweek()',
    'date_col_0.dayofyear()',
    'date_col_0.dayofmonth()',
    'date_col_0.weekofyear()',
    'date_col_0.month()',
    'date_col_0.quarter()',
    'date_col_0.year()',
    'datetime_col_0.timestamp_diff(datetime_col_1)',
    'date_col_0.date_diff(date_col_1)',
    'date_col_1.base_Sunday()',
]

grouped_expressions = [
    'x.sum()',
    '(1).sum()',
    '_ngroup()',
    'x.max()',
    'x.mean()',
    'x.median()',
    'x.min()',
    'x.nunique()',
    'x.size()',
    'z.count()',
    '_size()',
    '_count()',
]

project_expressions = [
    'x.sum()',
    '(1).sum()',
    'a.all()',
    'a.any()',
    'x.max()',
    'x.mean()',
    'x.median()',
    'x.min()',
    'x.nunique()',
    'x.size()',
    'x.std()',
    'x.var()',
    'z.count()',
    '_size()',
]

windowed_expressions = [
    'z.bfill()',
    'z.ffill()',
    'x.first()',
    'x.last()',
    'x.rank()',
    'x.cumprod()',
    'x.cumsum()',
    'z.cumcount()',
    'x.cummax()',
    'x.cummin()',
    'x.shift()',
    '_row_number()',
]

u_expressions = [  # not simply checkable as output varies
    '_uniform()',
]

up_expressions = [  # not simply checkable as output varies
    'x.any_value()',
]

TODO: windowed project for first/last?

In [18]:
e_expectations = [(parse(exp).op, 'e', exp, f(exp), f(exp).transform(d)) for exp in expressions]
g_expectations = [(parse(exp).op, 'g', exp, fg(exp), fg(exp).transform(d)) for exp in grouped_expressions]
p_expectations = [(parse(exp).op, 'p', exp, fp(exp), fp(exp).transform(d)) for exp in project_expressions]
w_expectations = [(parse(exp).op, 'w', exp, fw(exp), fw(exp).transform(d)) for exp in windowed_expressions]
u_results = (
    [(parse(exp).op, 'u', exp, f(exp), f(exp).transform(d)) for exp in u_expressions]
    + [(parse(exp).op, 'up', exp, fp(exp), fp(exp).transform(d)) for exp in up_expressions]
)

In [19]:
with gzip.open('expr_expectations.pkl.gz', 'wb') as out_f:
    pickle.dump(
        {
            'd': d,
            'expectations': e_expectations + g_expectations + p_expectations + w_expectations,
            'u_results': u_results,
        },
        out_f)

In [20]:
ops_list = e_expectations + g_expectations + p_expectations + w_expectations + u_results

op_catalog = pd.DataFrame({
    'op': [op for op, op_class, exp, ops, expect in ops_list],
    'expression': [exp for op, op_class, exp, ops, expect in ops_list],
    'op_class': [op_class for op, op_class, exp, ops, expect in ops_list],
})
op_catalog['Pandas'] = 'y'


def test_on_db(db_handle, *, msg_file):
    # test on db
    res_vector = ['?'] * len(ops_list)
    db_handle.insert_table(d, table_name='d', allow_overwrite=True)
    for i in range(len(ops_list)):
        op = ops_list[i][0]
        op_class = ops_list[i][1]
        exp = ops_list[i][2]
        ops = ops_list[i][3]
        expect = ops_list[i][4]
        try:
            res = db_handle.read_query(ops)
            if op_class not in  {'u', 'up'}:
                if data_algebra.test_util.equivalent_frames(res, expect):
                    res_vector[i] = 'y'
                else:
                    res_vector[i] = 'w'
                    print("", file=msg_file)
                    print("difference (w)",
                          file=msg_file)
                    print(f'op: {op}, op_class: {op_class}, example expression: {exp}, db: {db_handle.db_model}',
                          file=msg_file)
                    print("Pandas result (expectation):",
                          file=msg_file)
                    print(expect,
                          file=msg_file)
                    print("DB result:",
                          file=msg_file)
                    print(res,
                          file=msg_file)
                    print("query",
                          file=msg_file)
                    print(db_handle.to_sql(ops),
                          file=msg_file)
                    print("", file=msg_file)
            else:
                res_vector[i] = 'y'
        except Exception as ex:
            res_vector[i] = 'n'
            print("", file=msg_file)
            print("error (n)",
                  file=msg_file)
            print(f'op: {op}, op_class: {op_class}, example expression: {exp}, db: {db_handle.db_model}',
                  file=msg_file)
            print(f"caught: {ex}",
                  file=msg_file)
            print("", file=msg_file)
    db_handle.drop_table('d')
    return res_vector


db_handles = [
    data_algebra.SQLite.example_handle(),
    data_algebra.BigQuery.example_handle(),
    data_algebra.PostgreSQL.example_handle(),
    data_algebra.SparkSQL.example_handle(),
    data_algebra.MySQL.example_handle(),
]

22/01/18 18:53:27 WARN Utils: Your hostname, JAMiMac.local resolves to a loopback address: 127.0.0.1; using 192.168.0.155 instead (on interface en1)
22/01/18 18:53:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/18 18:53:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [21]:
warnings.filterwarnings("ignore")  # don't trigger our own warnings on old defs
with open("error_msgs.txt", 'w') as msg_file:
    for db_handle in db_handles:
        db_test_res = test_on_db(db_handle, msg_file=msg_file)
        op_catalog[str(db_handle.db_model)] = db_test_res
        db_handle.close()
warnings.filterwarnings("default")  # put back to default

22/01/18 18:56:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/01/18 18:56:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/johnmount/opt/anaconda3/envs/ai_academy_3_9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/7f/sdjycp_d08n8wwytsbgwqgsw0000gn/T/ipykernel_17791/1979511716.py", line 4, in <module>
    db_test_res = test_on_db(db_handle, msg_file=msg_file)
  File "/var/folders/7f/sdjycp_d08n8wwytsbgwqgsw0000gn/T/ipykernel_17791/4025021899.py", line 22, in test_on_db
    res = db_handle.read_query(ops)
  File "/Users/johnmount/Documents/work/data_algebra/data_algebra/db_model.py", line 2240, in read_query
    return self.db_model.read_query(conn=self.conn, q=q)
  File "/Users/johnmount/Documents/work/data_algebra/data_algebra/BigQuery.py", line 155, in read_query
    r = self.local_data_model.pd.DataFrame(conn.query(q).result().to_dataframe())
  File "/Users/johnmount/opt/anaconda3/envs/ai_academy_3_9/lib/python3.9/site-packages/google/cloud/bi

TypeError: object of type 'NoneType' has no len()

In [ ]:
op_catalog = op_catalog.sort_values(by=['op_class', 'op', 'expression'], inplace=False).reset_index(
    drop=True, inplace=False)
op_catalog['version'] = data_algebra.__version__
op_catalog

,op,expression,op_class,Pandas,SQLiteModel,BigQueryModel,PostgreSQLModel,SparkSQLModel,MySQLModel,version
0,!=,x != y,e,y,y,y,y,y,y,1.3.1
1,%,row_id % q,e,y,y,y,y,y,y,1.3.1
2,*,x * y,e,y,y,y,y,y,y,1.3.1
3,**,x ** y,e,y,y,y,y,y,y,1.3.1
4,+,x + y,e,y,y,y,y,y,y,1.3.1
...,...,...,...,...,...,...,...,...,...,...
115,ffill,z.ffill(),w,y,n,n,n,n,n,1.3.1
116,first,x.first(),w,y,n,n,n,y,n,1.3.1
117,last,x.last(),w,y,n,n,n,w,n,1.3.1
118,rank,x.rank(),w,y,n,n,n,y,n,1.3.1


In [ ]:
op_catalog.to_csv('op_catalog.csv', index=False)

In [ ]:
table_as_python = data_algebra.util.pandas_to_example_str(op_catalog)
table_as_python = pretty_format_python(table_as_python)

with open('op_catalog.py', 'w') as f_out:
    print("""

import data_algebra

pd = data_algebra.default_data_model.pd

    """,
          file=f_out)
    print("methods_table = " + table_as_python,
          file=f_out)

  * [data algebra](https://github.com/WinVector/data_algebra)
  * [data_algebra_catalog.ipynb](https://github.com/WinVector/data_algebra/blob/main/Examples/Methods/data_algebra_catalog.ipynb)
  * [op_cataglog.csv](https://github.com/WinVector/data_algebra/blob/main/Examples/Methods/op_catalog.csv)
  * [error_msgs.txt](https://github.com/WinVector/data_algebra/blob/main/Examples/Methods/error_msgs.txt)
  * [Method Warnings](https://github.com/WinVector/data_algebra/blob/main/Examples/Methods/MethodWarnings.ipynb)